In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
data = pd.read_csv('data.csv')
data = data.drop(columns=['id', 'Unnamed: 32'])
data.dropna()
display(data)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[data.columns.difference(['diagnosis'])], data['diagnosis'] ,test_size=0.2, random_state=42, stratify=data['diagnosis'])

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns=data.columns.difference(['diagnosis']))

In [ ]:
parameters = {'n_estimators':[50, 100, 200], 'max_depth':[3, 5, 7]}
model = RandomForestClassifier(random_state=42)
gsCV = GridSearchCV(model, parameters, cv=5)
gsCV.fit(X_train, y_train)
best_score_rfc = gsCV.best_score_
best_paramsf = gsCV.best_params_
print(best_score_rfc)
print(best_paramsf)

0.9626373626373628
{'max_depth': 5, 'n_estimators': 100}


In [ ]:
parameters = {'learning_rate':[0.1, 0.05, 0.01 ,0.5], 'n_estimators':[50, 100, 200]}
model = AdaBoostClassifier(random_state=42)
gsCV = GridSearchCV(model, parameters, cv=5)
gsCV.fit(X_train, y_train)
best_score_Ada = gsCV.best_score_
best_paramsa = gsCV.best_params_
print(best_score_Ada)
print(best_paramsa)

0.9582417582417584
{'learning_rate': 0.5, 'n_estimators': 200}


In [ ]:
X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test, columns=data.columns.difference(['diagnosis']))

In [ ]:
rfc = RandomForestClassifier(max_depth=best_paramsf['max_depth'], n_estimators=best_paramsf['n_estimators'], random_state=42)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='M')
recall = recall_score(y_test, y_pred, pos_label='M')
f1 = f1_score(y_test, y_pred, pos_label='M')
print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall:", recall)
print("F1-score:", f1)
report = classification_report(y_test, y_pred, zero_division=1)
print(report)

Acurácia: 0.9736842105263158
Precisão: 1.0
Recall: 0.9285714285714286
F1-score: 0.962962962962963
              precision    recall  f1-score   support

           B       0.96      1.00      0.98        72
           M       1.00      0.93      0.96        42

    accuracy                           0.97       114
   macro avg       0.98      0.96      0.97       114
weighted avg       0.97      0.97      0.97       114



No caso vemos que ele teve uma precisão de 100% para acertar os casos que era a label era 'M' e uma precisão de 96% em acertar os casos que label era 'B'.

In [ ]:
importances = rfc.feature_importances_
feature_names = [f"{data.columns[i]}" for i in range(len(importances))]
feature_importances = list(zip(feature_names, importances))
feature_importances_sorted = sorted(feature_importances, key=lambda x: x[1], reverse=True)
for feature, importance in feature_importances_sorted:
    print(f"{feature}: {importance}")

texture_mean: 0.14453254454647757
fractal_dimension_se: 0.11202242716595961
compactness_mean: 0.10990652887734599
concavity_se: 0.10733440058276401
concave points_mean: 0.09239754020601075
diagnosis: 0.07544462875889284
smoothness_se: 0.060879390927131606
radius_mean: 0.04836607825494155
concave points_se: 0.03486889933929863
symmetry_mean: 0.027644717339381647
radius_se: 0.020230748941480664
symmetry_worst: 0.020163212502918446
compactness_worst: 0.01842383880784586
concavity_worst: 0.015849171779416995
symmetry_se: 0.015831764210772797
smoothness_mean: 0.015748434298395328
perimeter_worst: 0.01437046518143776
perimeter_mean: 0.011652825782333199
compactness_se: 0.009149581799838609
radius_worst: 0.007773900419537414
perimeter_se: 0.0054687117336740376
texture_se: 0.004801220680702004
area_mean: 0.004665253996467623
fractal_dimension_mean: 0.004069750919123961
area_se: 0.0036205723035647413
area_worst: 0.0035093033013499594
concave points_worst: 0.0034973114362193474
smoothness_worst:

In [ ]:
feature_importances_sorted = list(filter(lambda x: x[0] != 'diagnosis', feature_importances_sorted))
mais_importantes = [tup[0] for tup in feature_importances_sorted[:10]]
rfc_re = RandomForestClassifier(max_depth=best_paramsf['max_depth'], n_estimators=best_paramsf['n_estimators'], random_state=42)
rfc_re.fit(X_train.loc[:,mais_importantes], y_train)
y_pred = rfc_re.predict(X_test.loc[:, mais_importantes])
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='M')
recall = recall_score(y_test, y_pred, pos_label='M')
f1 = f1_score(y_test, y_pred, pos_label='M')
print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall:", recall)
print("F1-score:", f1)
report = classification_report(y_test, y_pred, zero_division=1)
print(report)

Acurácia: 0.9298245614035088
Precisão: 0.9473684210526315
Recall: 0.8571428571428571
F1-score: 0.9
              precision    recall  f1-score   support

           B       0.92      0.97      0.95        72
           M       0.95      0.86      0.90        42

    accuracy                           0.93       114
   macro avg       0.93      0.91      0.92       114
weighted avg       0.93      0.93      0.93       114



A remoção de features implicou na redução da precisão do modelo. Antes ele acertava 100% das label 'M' e agora acerta 95%, assim como também reduziu a precisão da label 'B', ou seja uma redução geral na precisão do modelo